<a href="https://colab.research.google.com/github/Jepees/StepAhead-Nuevo/blob/main/scripts/2_data_merger_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
import re

In [ ]:
brands = ['910', 'Ortuseight', 'Mills', 'Nuevo']

datas = {
            'df_910'        : pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/Tokopedia_review_910.csv"),
            'df_ortuseight' : pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/Tokopedia_review_Ortuseight.csv"),
            'df_mills'      : pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/Tokopedia_review_Mills.csv"),
            'df_nuevo'      : pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/Tokopedia_review_Nuevo.csv"),
        }
data_map = pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/database_sepatu.csv")

for brand in brands:
    datas[f'df_{brand.lower()}']['Brand'] = brand

In [ ]:
df = pd.concat(list(datas.values()), ignore_index=True)
print(df.shape)
df.sample(5)

(13922, 10)


,nama barang,kategori,harga,date,user,varian,rating,review,bantu,Brand
10982,MILLS SEPATU SEPAKBOLA XYCLOPS HELIOS FG JR BE...,bola,Rp359.200,1 bulan lalu,H***a,BEIGE/BRIGH.RED - 34,bintang 5,keren,0 orang terbantu,Mills
7755,Ortuseight Sepatu Sneakers Bristol 1.2 Ash Gre...,sneakers,Rp599.000,2 bulan lalu,F***n,NaN,bintang 5,"worth it, ty ortus😍",0 orang terbantu,Ortuseight
912,910 NINETEN HAZE PRO sepatu lari - All Black R...,Lari,Rp1.424.905,5 bulan lalu,v***c,39.0,bintang 5,"Harus nya ngambil sz 40 sih, di 39 ngepres tpi...",0 orang terbantu,910
11110,MILLS Sepatu Sportage CL Black/White 9701001 -...,NaN,Rp299.000,5 bulan lalu,Q***q,Black/White - 44,bintang 5,Bagus sesuai dengan pesanan,0 orang terbantu,Mills
5868,Ortuseight Sepatu Running Hyperfuse 3.0 Electr...,Lari,Rp799.000,1 bulan lalu,J***i,Electricity Vermilion - 40,bintang 5,Good,1 orang terbantu,Ortuseight


In [ ]:
# Merge berdasarkan 'nama barang' agar dapat 'nama fix' dan kategori bersih
df = df.merge(
    data_map[['nama barang', 'nama fix', 'kategori']],
    on='nama barang',
    how='left',
    suffixes=('', '_baru')
)

df['nama barang'] = df['nama fix']
df['kategori'] = df['kategori_baru']


df = df.drop(columns=['nama fix', 'kategori_baru'])

df

,nama barang,kategori,harga,date,user,varian,rating,review,bantu,Brand
0,nineten haze strike prime,sepatu lari,Rp550.905,2 hari lalu,A***s,42.0,bintang 5,Sepatunya ok dan pas dengan ukuran.. Nyampenya...,0 orang terbantu,910
1,nineten haze strike prime,sepatu lari,Rp550.905,1 hari lalu,Z***i,42.0,bintang 5,barang original enak di pakek ngebut sepatunya,0 orang terbantu,910
2,nineten haze strike prime,sepatu lari,Rp550.905,1 minggu lalu,R***i,45.0,bintang 5,Bagus nyaman melebihi ekspektasi saya,0 orang terbantu,910
3,nineten haze strike prime,sepatu lari,Rp550.905,2 minggu lalu,V***u,41.0,bintang 5,Produknya mantapppplll,1 orang terbantu,910
4,nineten haze strike prime,sepatu lari,Rp550.905,3 hari lalu,Y***e,40.0,bintang 5,Pengiriman cepat,0 orang terbantu,910
...,...,...,...,...,...,...,...,...,...,...
13917,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,Lebih dari 1 tahun lalu,A***e,38,bintang 4,NaN,0 orang terbantu,Nuevo
13918,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,Lebih dari 1 tahun lalu,D***i,40,bintang 5,NaN,0 orang terbantu,Nuevo
13919,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,Lebih dari 1 tahun lalu,d***a,42,bintang 5,Lumyan,0 orang terbantu,Nuevo
13920,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,Lebih dari 1 tahun lalu,r***n,41,bintang 3,NaN,0 orang terbantu,Nuevo


In [ ]:
for col in df.columns:
    df[col] = df[col].astype("str")

In [ ]:
# menghapus baris duplikat

print(f'data duplikat sebelum penghapusan: {df.duplicated().sum()}')
df.drop_duplicates(inplace=True)
print(f'data duplikat setelah penghapusan: {df.duplicated().sum()}')

data duplikat sebelum penghapusan: 9
data duplikat setelah penghapusan: 0


In [ ]:
print(df.shape)
df.sample(5)

(13913, 10)


,nama barang,kategori,harga,date,user,varian,rating,review,bantu,Brand
12548,nuevo velocity,sepatu lari,Rp156.350,5 bulan lalu,A***7,Hitam Gold - 41,bintang 5,Mantaap 👍👍,0 orang terbantu,Nuevo
8098,ortuseight forrest v1,sepatu lifestyle,Rp699.000,1 minggu lalu,I***u,Petrichor - 40,bintang 2,menurutku kualitas kurang dengan harga 600rb+ ...,0 orang terbantu,Ortuseight
3082,nineten takashi run 1.5,sepatu lari,Rp645.905,10 bulan lalu,D***k,43.0,bintang 5,Bagus,0 orang terbantu,910
8840,ortuseight shkhara,sepatu lari,Rp1.199.000,5 bulan lalu,C***l,Forest Green - 45,bintang 5,"Pake di muddy trail, enak juga! Sol nya cukup ...",0 orang terbantu,Ortuseight
4870,ortuseight trofeo in,sepatu futsal,Rp349.000,8 bulan lalu,Paul,44,bintang 5,Sip mantap. dirasain si keliatan bagus ya. Uku...,0 orang terbantu,Ortuseight


In [ ]:
def ke_bulan_tahun(text):
    today = datetime.today()

    if not isinstance(text, str):
        return None
    text = text.lower().strip()
    tanggal = today

    if "hari ini" in text or "hari lalu" in text or "minggu lalu" in text:
        tanggal = today
    elif "bulan lalu" in text:
        jumlah = int(re.search(r"\d+", text).group())
        tanggal = today - timedelta(days=30 * jumlah)  # asumsi 30 hari per bulan
    elif "lebih dari 1 tahun" in text:
        tanggal = today - timedelta(days=450)
    elif "tahun lalu" in text:
        tanggal = today - timedelta(days=365)
    else:
        return None
    return tanggal.strftime('%B %Y')

def pisah_varian(text):
    if isinstance(text, str):
        if '-' in text:
            parts = text.split('-', maxsplit=1)
            varian = parts[0].strip()
            ukuran = parts[1].strip()
        else:
            if text.strip().isdigit():
                varian = None
                ukuran = text.strip()
            else:
                varian = text.strip()
                ukuran = None
    else:
        varian = None
        ukuran = None
    return pd.Series([varian, ukuran])

def pisah_varian(text):
    def proses_ukuran(uk):
        uk = uk.strip()
        if '/' in uk:
            uk = uk.split('/')[0].strip()
        if uk.replace('.', '', 1).isdigit():
            return str(int(float(uk)))  # bulatkan ke bawah
        return None

    varian, ukuran = None, None

    if isinstance(text, str):
        text = text.strip()
        if '-' in text:
            parts = text.split('-', maxsplit=1)
            part0 = parts[0].strip()
            part1 = parts[1].strip()

            is_part0_num = part0.replace('.', '', 1).replace('/', '', 1).isdigit()
            is_part1_num = part1.replace('.', '', 1).replace('/', '', 1).isdigit()

            if is_part0_num:
                ukuran = proses_ukuran(part0)
                varian = part1
            elif is_part1_num:
                varian = part0
                ukuran = proses_ukuran(part1)
            else:
                varian = f"{part0}-{part1}"
        else:
            # Kasus tanpa strip
            if text.replace('.', '', 1).replace('/', '', 1).isdigit():
                ukuran = proses_ukuran(text)
                varian = None
            else:
                varian = text
                ukuran = None

    return pd.Series([varian, ukuran], index=['varian_baru', 'ukuran'])

In [ ]:
df['date'] = df['date'].apply(ke_bulan_tahun)
df[['varian', 'ukuran']] = df['varian'].apply(pisah_varian)
# df['ukuran'] = pd.to_numeric(df['ukuran'], errors='coerce').astype('Int64')


df['rating'] = (
    df['rating']
    .str.replace('bintang ', '', regex=False)
    .astype('Int64')
)

df['harga_int'] = (
    df['harga']
    .str.replace('Rp', '', regex=False)      # hilangkan "Rp"
    .str.replace('.', '', regex=False)       # hilangkan titik pemisah ribuan
    .str.strip()                             # hilangkan spasi ekstra
    .astype(int)                             # ubah ke integer
)

df['terbantu'] = (
    df['bantu']
    .str.replace(' orang terbantu', '', regex=False)
    .astype('Int64')
)

print(df.shape)
df.sample(5)

(13913, 13)


,nama barang,kategori,harga,date,user,varian,rating,review,bantu,Brand,ukuran,harga_int,terbantu
12981,nuevo wilmax,sepatu lari,Rp157.500,June 2025,U***s,Biru Tosca/Oren,4,Dg harga segitu wajar sepatu lumayan smg lemny...,0 orang terbantu,Nuevo,40,157500,0
1111,nineten irezumi spacepunk,sepatu lari,Rp645.905,October 2024,T***S,None,5,Mntap👍👍,0 orang terbantu,910,42,645905,0
3013,nineten nakami swish,sepatu basket,Rp598.405,December 2024,W***H,None,5,"Bagus utk yg suka basket , bunyi cit cit 🤣🤙",0 orang terbantu,910,43,598405,0
6916,ortuseight quadra fg,sepatu bola,Rp399.000,January 2025,T***a,Cloud Blue Black,5,Bentuk dan ukuran: Bagus dan pas,0 orang terbantu,Ortuseight,41,399000,0
3138,nineten takashi run 1.5,sepatu lari,Rp645.905,February 2025,T***S,None,5,Mantap,0 orang terbantu,910,44,645905,0


In [ ]:
df = df[[   'Brand', 'nama barang', 'kategori', 'harga', 'harga_int',
            'user', 'varian', 'ukuran',
            'date', 'review', 'rating', 'terbantu']]

print(df.shape)
df.sample(5)

(13913, 12)


,Brand,nama barang,kategori,harga,harga_int,user,varian,ukuran,date,review,rating,terbantu
7504,Ortuseight,ortuseight berlin,sepatu lifestyle,Rp599.000,599000,M***m,Walnut Fern Green,43,May 2025,Bagus sekali,5,0
11645,Nuevo,nuevo crosspeed,sepatu lari,Rp153.600,153600,a****,Putih Merah Biru,43,November 2024,dari segi ke empukan si lumayan tapi di mines ...,4,0
1734,910,nineten reishika bolster,sepatu lari,Rp569.905,569905,F***r,None,44,September 2024,Beli karena suka desainnya yang agresif. Saya ...,5,0
7147,Ortuseight,ortuseight sophon fg,sepatu bola,Rp379.000,379000,j***e,White Cyan,42,March 2025,Bentuk dan ukuran: Sesuai,5,0
13889,Nuevo,nuevo soars xi,sepatu lari,Rp181.500,181500,Q***y,STABILO KUNING,42,June 2025,mantapp,5,0


1. tren pembelian 4 brand, fokus ke sepatu nuevo
2. sentimen produk2 nuevo
3. negatif ke arah apanya?
4. topic di sentimen positif di brand lain
5. sepatu 5-10 sepatu paling populer di brand lain (tanpa nuevo)
6. kenapa itu populer?
7. rata2 harga per kategori sepatu antara masing2 brand

In [ ]:
df.loc[df['nama barang'].isna(), 'nama barang'] = 'nineten easy train'
df.loc[df['kategori'].isna(), 'kategori'] = 'sepatu lari'

In [ ]:
for col in df.columns:
    print(f'kolom {col}: {df[col].isna().sum()}')

kolom Brand: 0
kolom nama barang: 0
kolom kategori: 0
kolom harga: 0
kolom harga_int: 0
kolom user: 0
kolom varian: 3947
kolom ukuran: 182
kolom date: 0
kolom review: 0
kolom rating: 0
kolom terbantu: 0


In [ ]:
# df.to_csv('data_clean_merge.csv', index=False)
df

,Brand,nama barang,kategori,harga,harga_int,user,varian,ukuran,date,review,rating,terbantu
0,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,A***s,None,42,July 2025,Sepatunya ok dan pas dengan ukuran.. Nyampenya...,5,0
1,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,Z***i,None,42,July 2025,barang original enak di pakek ngebut sepatunya,5,0
2,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,R***i,None,45,July 2025,Bagus nyaman melebihi ekspektasi saya,5,0
3,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,V***u,None,41,July 2025,Produknya mantapppplll,5,1
4,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,Y***e,None,40,July 2025,Pengiriman cepat,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13917,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,A***e,None,38,April 2024,nan,4,0
13918,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,D***i,None,40,April 2024,nan,5,0
13919,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,d***a,None,42,April 2024,Lumyan,5,0
13920,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,r***n,None,41,April 2024,nan,3,0
